In [1]:
import urllib.parse
from pathlib import Path

import psycopg
from psycopg.rows import dict_row
from tqdm.auto import tqdm

from fishsense_data_processing_spider.backend import get_project_export
from fishsense_data_processing_spider.config import PG_CONN_STR, settings

In [2]:
export = get_project_export(
    project_id=10,
    label_studio_api_key=settings.label_studio.api_key,
    label_studio_host=settings.label_studio.host
)

In [3]:
image_paths = {task['id']: Path(urllib.parse.parse_qs(urllib.parse.urlparse(task['data']['img']).query)['d'][0]).relative_to(
    'fs_png_labeling_project_laser/REEF/data').with_suffix('.ORF').as_posix() for task in export}

In [4]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    cur.execute('SELECT path, image_md5 FROM images;')
    result = cur.fetchall()

    cksum_lookup = {row['path']:row['image_md5'] for row in result}
    task_id__cksums = {task_id:cksum_lookup[path] for task_id, path in image_paths.items() if path in cksum_lookup}

    cur.executemany(
        'INSERT INTO laser_labels (cksum, task_id) VALUES (%(cksum)s, %(task_id)s) ON CONFLICT DO NOTHING;',
        [
            {
                'cksum': cksum,
                'task_id': task_id
            }
            for task_id, cksum in task_id__cksums.items()
        ]
    )
    con.commit()

In [5]:
bad_paths = [path for path in image_paths.values() if path not in cksum_lookup]

In [6]:
coords = {task['id']: (int(task['annotations'][0]['result'][0]['value']['x'] / 100 *
                           task['annotations'][0]['result'][0]['original_width']),
                       int(task['annotations'][0]['result'][0]['value']['y'] / 100 *
                           task['annotations'][0]['result'][0]['original_height']))
          for task in export
          if len(task['annotations']) > 0 and len(task['annotations'][0]['result']) > 0}

In [7]:
params_seq = [
    {
        'task_id': task_id,
        'x': coord[0],
        'y': coord[1]
    }
    for task_id, coord in coords.items()
]

In [8]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    cur.executemany(
        'UPDATE laser_labels SET x = %(x)s, y = %(y)s WHERE task_id = %(task_id)s;',
        params_seq
    )
    con.commit()

In [9]:
export = get_project_export(
    project_id=39,
    label_studio_api_key=settings.label_studio.api_key,
    label_studio_host=settings.label_studio.host
)

In [10]:
param_seq = [{
                'cksum': Path(urllib.parse.urlparse(task['data']['img']).path).stem,
                'x': int(task['annotations'][0]['result'][0]['value']['x'] / 100 *
                        task['annotations'][0]['result'][0]['original_width']),
                'y': int(task['annotations'][0]['result'][0]['value']['y'] / 100 *
                        task['annotations'][0]['result'][0]['original_height'])
            }
            for task in export
            if len(task['annotations']) > 0 and len(task['annotations'][0]['result']) > 0
            ]

In [11]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    cur.executemany(
        'UPDATE laser_labels SET x = %(x)s, y = %(y)s WHERE cksum = %(cksum)s;',
        param_seq
    )
    con.commit()